скип этой хуйни, чекай ласт ячейку(это на всякий случай оставил вдруг поможет)

In [ ]:
!pip install torch torchvision
!pip install transformers datasets accelerate
!pip install Pillow
!pip install huggingface_hub

In [ ]:
# @title Шаг 1: Установка зависимостей и проверка GPU (ФИНАЛЬНАЯ ВЕРСИЯ)
!pip install -q torch torchvision
!pip install -q transformers datasets accelerate Pillow huggingface_hub

import torch
print(f"Используемое устройство: cuda" if torch.cuda.is_available() else "Используемое устройство: cpu")


Используемое устройство: cpu


In [ ]:
# dataloaders.py
# @title Шаг 3: Определяем классы Dataloader'ов (Исправленная версия)
import os
import json
from PIL import Image
from torch.utils.data import Dataset
from datasets import load_dataset

class SlakeDataset(Dataset):
    """
    Даталоадер для датасета SLAKE с Hugging Face.
    Фильтрует данные, оставляя только англоязычные вопросы.
    """
    def __init__(self, processor, split='train'):
        print(f"Loading SLAKE dataset for split: {split}")
        self.processor = processor
        full_dataset = load_dataset("BoKelvin/SLAKE", split=split)

        # --- ИСПРАВЛЕНИЕ ЗДЕСЬ ---
        # Фильтруем по столбцу 'q_lang', как вы и указали.
        self.dataset = full_dataset.filter(lambda example: example['q_lang'] == 'en')

        print(f"Original size: {len(full_dataset)}, Filtered size (en only): {len(self.dataset)}")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image'].convert("RGB")
        question = item['question']
        answer = item['answer']
        processed_image = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)
        return {"image": processed_image, "question": question, "answer": answer}


class VQARADDataset(Dataset):
    """
    Даталоадер для датасета VQA-RAD (скачанного вручную).
    """
    def __init__(self, processor, root_dir, split='train'):
        print(f"Loading VQA-RAD dataset for split: {split}")
        self.processor = processor
        self.root_dir = root_dir
        self.image_dir = os.path.join(self.root_dir, 'images')
        json_path = os.path.join(self.root_dir, f'{split}.json')
        with open(json_path, 'r') as f:
            self.dataset = json.load(f)
        print(f"Dataset size: {len(self.dataset)}")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_path = os.path.join(self.image_dir, item['image_name'])
        image = Image.open(image_path).convert("RGB")
        question = item['question']
        answer = item['answer']
        processed_image = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)
        return {"image": processed_image, "question": question, "answer": answer}



In [ ]:
# @title Шаг 4: Определяем архитектуру модели (Vision + MLP) (ФИНАЛЬНАЯ ВЕРСИЯ)
import torch.nn as nn
from transformers import AutoModel, CLIPVisionModel

class VisionMLP(nn.Module):
    """
    Финальная версия модели, которая корректно загружает ОБА типа энкодеров,
    используя официальные методы.
    """
    def __init__(self, encoder_path, mlp_embedding_dim=4096):
        super().__init__()
        self.encoder_path = encoder_path

        # --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: УМНЫЙ ЗАГРУЗЧИК ---
        if "microsoft/BiomedCLIP" in encoder_path:
            self.model_type = 'biomed_clip'
            print("Detected BiomedCLIP (custom model). Loading with `trust_remote_code=True`.")
            # Это ОФИЦИАЛЬНЫЙ способ загрузки этой модели.
            # Он выполняет код из репозитория модели для ее правильной сборки.
            self.vision_model = AutoModel.from_pretrained(encoder_path, trust_remote_code=True)
            # У этой модели скрытое состояние 768
            vision_hidden_size = 768
            print("Custom BiomedCLIP model loaded successfully.")
        else:
            self.model_type = 'standard_clip'
            print(f"Detected standard Transformers model: {encoder_path}")
            # Для стандартных моделей, как pubmed-clip, загружаем только vision-часть
            self.vision_model = CLIPVisionModel.from_pretrained(encoder_path)
            vision_hidden_size = self.vision_model.config.hidden_size
            print("Standard CLIP vision model loaded successfully.")

        # MLP-проектор остается без изменений
        self.mlp = nn.Sequential(
            nn.Linear(vision_hidden_size, vision_hidden_size * 4),
            nn.GELU(),
            nn.Linear(vision_hidden_size * 4, mlp_embedding_dim)
        )
        print(f"MLP configured: {vision_hidden_size} -> {mlp_embedding_dim}")

    def forward(self, pixel_values):
        # --- УМНЫЙ FORWARD PASS ---
        if self.model_type == 'biomed_clip':
            # У этой кастомной модели есть свой метод для получения эмбеддингов изображений
            image_features = self.vision_model.get_image_features(pixel_values=pixel_values)
        else: # standard_clip
            # Стандартная модель возвращает объект вывода, из которого мы берем pooler_output
            outputs = self.vision_model(pixel_values=pixel_values)
            image_features = outputs.pooler_output

        return self.mlp(image_features)

print("Класс VisionMLP (ФИНАЛЬНАЯ ВЕРСИЯ) определен. Теперь можно запускать Шаг 5.")


Класс VisionMLP (ФИНАЛЬНАЯ ВЕРСИЯ) определен. Теперь можно запускать Шаг 5.


In [ ]:
# @title Шаг 5: Запуск основного пайплайна (ФИНАЛЬНАЯ ВЕРСИЯ)
import os
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoProcessor

# --- Переопределяем даталоадеры здесь для простоты ---
class SlakeDataset(Dataset):
    def __init__(self, processor, split='train'):
        self.processor = processor
        full_dataset = load_dataset("BoKelvin/SLAKE", split=split)
        self.dataset = full_dataset.filter(lambda example: example['q_lang'] == 'en')
    def __len__(self): return len(self.dataset)
    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image'].convert("RGB")
        # Процессор возвращает dict, нам нужны 'pixel_values'
        processed = self.processor(images=image, return_tensors="pt")
        return {"image": processed['pixel_values'].squeeze(0), "question": item['question'], "answer": item['answer']}

class VQARADDataset(Dataset):
    def __init__(self, processor, root_dir, split='train'):
        self.processor = processor
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, 'images')
        with open(os.path.join(root_dir, f'{split}.json'), 'r') as f: self.dataset = json.load(f)
    def __len__(self): return len(self.dataset)
    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = Image.open(os.path.join(self.image_dir, item['image_name'])).convert("RGB")
        processed = self.processor(images=image, return_tensors="pt")
        return {"image": processed['pixel_values'].squeeze(0), "question": item['question'], "answer": item['answer']}

# --- Конфигурация запуска ---
ENCODER_CHOICE = 'biomedclip' # 'biomedclip' или 'pubmedclip'
DATASET_CHOICE = 'slake'      # 'slake' или 'vqa_rad'

def main():
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_PATHS = {
        'biomedclip': 'microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224',
        'pubmedclip': 'flaviagiammarino/pubmed-clip-vit-base-patch32'
    }
    ENCODER_PATH = MODEL_PATHS[ENCODER_CHOICE]

    print(f"--- Running pipeline with {ENCODER_CHOICE} on {DATASET_CHOICE} dataset ---")
    print(f"Using device: {DEVICE}")

    # 1. Загрузка препроцессора. AutoProcessor универсален.
    # `trust_remote_code=True` нужен для BiomedCLIP, но не повредит и pubmedclip.
    processor = AutoProcessor.from_pretrained(ENCODER_PATH, trust_remote_code=True)

    # 2. Инициализация датасета
    if DATASET_CHOICE == 'slake': dataset = SlakeDataset(processor=processor, split='train')
    else: dataset = VQARADDataset(processor=processor, root_dir="data/vqa_rad/VQA_RAD_Dataset", split='train')

    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

    # 3. Инициализация модели. Теперь она сама все знает.
    model = VisionMLP(encoder_path=ENCODER_PATH, mlp_embedding_dim=4096).to(DEVICE)

    # Заморозка весов. `model.vision_model` теперь наш стандартизированный атрибут
    for param in model.vision_model.parameters():
        param.requires_grad = False

    # 4. Прогон одного батча
    print("\n--- Processing one batch ---")
    model.eval()
    with torch.no_grad():
        batch = next(iter(dataloader))
        images = batch['image'].to(DEVICE)
        image_embeddings = model(images)

        print(f"\nBatch loaded. Shape: {images.shape}")
        print(f"Output embeddings shape: {image_embeddings.shape}")
        assert image_embeddings.shape == (images.shape[0], 4096)
        print("\nShape check PASSED.")

    print("\n--- Pipeline execution successful! ---")

if __name__ == '__main__':
    main()


--- Running pipeline with biomedclip on slake dataset ---
Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Detected BiomedCLIP (custom model). Loading with `trust_remote_code=True`.


OSError: microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224 does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

прогоняем биоклип на слейке

In [23]:
# ==============================================================================
# КОД.
# ==============================================================================

# --- 1. УСТАНОВКА ЗАВИСИМОСТЕЙ ---
print("--- 1. Установка правильных зависимостей (включая open_clip_torch) ---")
!pip install -q torch torchvision torchaudio
!pip install -q transformers datasets accelerate Pillow huggingface_hub
!pip install -q open_clip_torch

import os
import json
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPVisionModel, AutoImageProcessor
from huggingface_hub import snapshot_download, hf_hub_download
import open_clip

# --- 2. DATALOADER ДЛЯ SLAKE ---
class SlakeDataset(Dataset):
    def __init__(self, image_processor, hf_repo_id='BoKelvin/SLAKE', split='train'):
        self.image_processor = image_processor
        self.root_dir = snapshot_download(repo_id=hf_repo_id, repo_type='dataset')
        img_dir_path = os.path.join(self.root_dir, 'imgs')
        if not os.path.exists(img_dir_path):
            os.system(f"unzip -q -o {os.path.join(self.root_dir, 'imgs.zip')} -d {self.root_dir}")
        json_path = os.path.join(self.root_dir, f"{split}.json")
        with open(json_path, 'r') as f:
            full_data = json.load(f)
        self.dataset = [item for item in full_data if item['q_lang'] == 'en']

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_name = item['img_name']
        image_path = os.path.join(self.root_dir, 'imgs', image_name)
        image = Image.open(image_path).convert("RGB")
        processed_image = self.image_processor(image)
        if processed_image.dim() > 3:
            processed_image = processed_image.squeeze(0)
        return {"image": processed_image, "question": item['question'], "answer": item['answer']}

# --- 3. КЛАСС МОДЕЛИ ---
class VisionMLP(nn.Module):
    def __init__(self, vision_encoder, encoder_output_dim, mlp_output_dim):
        super().__init__()
        self.vision_encoder = vision_encoder
        self.model_type = 'open_clip' if isinstance(vision_encoder, open_clip.model.VisionTransformer) else 'transformers'
        self.mlp = nn.Sequential(
            nn.Linear(encoder_output_dim, encoder_output_dim * 4), nn.GELU(),
            nn.Linear(encoder_output_dim * 4, mlp_output_dim))
        print(f"--- MLP сконфигурирован: {encoder_output_dim} -> {mlp_output_dim} ---")

    def forward(self, pixel_values):
        if self.model_type == 'open_clip':
            image_features = self.vision_encoder(pixel_values)
        else: # transformers
            image_features = self.vision_encoder(pixel_values=pixel_values).pooler_output
        return self.mlp(image_features)

# --- 4. ОСНОВНАЯ ФУНКЦИЯ ЗАПУСКА ---
def main():
    ENCODER_CHOICE = 'biomedclip' # 'biomedclip' или 'pubmedclip'
    MLP_OUTPUT_DIM = 4096
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_PATHS = {
        'biomedclip': 'microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224',
        'pubmedclip': 'flaviagiammarino/pubmed-clip-vit-base-patch32'
    }

    print(f"\n--- 4. Запускаю пайплайн: {ENCODER_CHOICE} на slake ---")

    if ENCODER_CHOICE == 'biomedclip':
        print("--- Загрузка BiomedCLIP... ---")
        model_obj, _, image_processor = open_clip.create_model_and_transforms('ViT-B-16', pretrained='openai')
        checkpoint_path = hf_hub_download(repo_id=MODEL_PATHS['biomedclip'], filename="open_clip_pytorch_model.bin")
        model_obj.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE), strict=False)
        vision_encoder = model_obj.visual

        # --- ФИНАЛЬНОЕ ИСПРАВЛЕНИЕ ЗДЕСЬ ---
        # Размерность выхода берется из матрицы проекции
        encoder_output_dim = model_obj.visual.proj.shape[1]

        print(f"--- BiomedCLIP загружен. Фактическая размерность выхода: {encoder_output_dim} ---")
    else: # pubmedclip
        print("--- Загрузка PubMedCLIP... ---")
        vision_encoder = CLIPVisionModel.from_pretrained(MODEL_PATHS['pubmedclip'])
        image_processor = AutoImageProcessor.from_pretrained(MODEL_PATHS['pubmedclip'])
        encoder_output_dim = vision_encoder.config.hidden_size
        print(f"--- PubMedCLIP загружен. Фактическая размерность выхода: {encoder_output_dim} ---")

    dataset = SlakeDataset(image_processor=image_processor)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

    model = VisionMLP(
        vision_encoder=vision_encoder,
        encoder_output_dim=encoder_output_dim,
        mlp_output_dim=MLP_OUTPUT_DIM
    ).to(DEVICE)

    for param in model.vision_encoder.parameters(): param.requires_grad = False

    print("\n--- 5. Обрабатываю один батч... ---")
    model.eval()
    with torch.no_grad():
        batch = next(iter(dataloader))
        images = batch['image'].to(DEVICE)
        image_embeddings = model(images)

        print(f"\n- Форма входного батча: {images.shape}")
        print(f"- Форма выходных эмбеддингов: {image_embeddings.shape}")
        assert image_embeddings.shape[1] == MLP_OUTPUT_DIM, "Размерность эмбеддинга неверна!"
        print("\n--- Проверка формы пройдена. ---")

    print("\n\n*** ПАЙПЛАЙН УСПЕШНО ВЫПОЛНЕН ***")

# --- 5. ЗАПУСК ---
if __name__ == '__main__':
    main()



--- 1. Установка правильных зависимостей (включая open_clip_torch) ---

--- 4. Запускаю пайплайн: biomedclip на slake ---
--- Загрузка BiomedCLIP... ---
--- BiomedCLIP загружен. Фактическая размерность выхода: 512 ---


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

--- MLP сконфигурирован: 512 -> 4096 ---

--- 5. Обрабатываю один батч... ---

- Форма входного батча: torch.Size([4, 3, 224, 224])
- Форма выходных эмбеддингов: torch.Size([4, 4096])

--- Проверка формы пройдена. ---


*** ПАЙПЛАЙН УСПЕШНО ВЫПОЛНЕН ***


прогоняем пубмед

In [25]:
# ==============================================================================
# КОД.
# ==============================================================================

# --- 1. УСТАНОВКА ЗАВИСИМОСТЕЙ ---
print("--- 1. Установка правильных зависимостей (включая open_clip_torch) ---")
!pip install -q torch torchvision torchaudio
!pip install -q transformers datasets accelerate Pillow huggingface_hub
!pip install -q open_clip_torch

import os
import json
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPVisionModel, AutoImageProcessor
from huggingface_hub import snapshot_download, hf_hub_download
import open_clip

# --- 2. DATALOADER ДЛЯ SLAKE (С ФИНАЛЬНЫМ ИСПРАВЛЕНИЕМ) ---
class SlakeDataset(Dataset):
    def __init__(self, image_processor, hf_repo_id='BoKelvin/SLAKE', split='train'):
        self.image_processor = image_processor
        self.root_dir = snapshot_download(repo_id=hf_repo_id, repo_type='dataset')
        img_dir_path = os.path.join(self.root_dir, 'imgs')
        if not os.path.exists(img_dir_path):
            os.system(f"unzip -q -o {os.path.join(self.root_dir, 'imgs.zip')} -d {self.root_dir}")
        json_path = os.path.join(self.root_dir, f"{split}.json")
        with open(json_path, 'r') as f:
            full_data = json.load(f)
        self.dataset = [item for item in full_data if item['q_lang'] == 'en']

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_name = item['img_name']
        image_path = os.path.join(self.root_dir, 'imgs', image_name)
        image = Image.open(image_path).convert("RGB")

        # Вызываем процессор. Для transformers он вернет объект, для open_clip - тензор
        processed_output = self.image_processor(images=image, return_tensors="pt")

        # --- КЛЮЧЕВОЕ ИСПРАВЛЕНИЕ ЗДЕСЬ ---
        # Унифицируем выход: всегда извлекаем тензор, если он внутри объекта
        if hasattr(processed_output, 'pixel_values'):
            processed_image = processed_output.pixel_values
        else:
            processed_image = processed_output

        # Убираем лишнюю размерность батча, которую добавляет процессор
        if processed_image.dim() > 3:
            processed_image = processed_image.squeeze(0)

        return {"image": processed_image, "question": item['question'], "answer": item['answer']}

# --- 3. КЛАСС МОДЕЛИ ---
class VisionMLP(nn.Module):
    def __init__(self, vision_encoder, encoder_output_dim, mlp_output_dim):
        super().__init__()
        self.vision_encoder = vision_encoder
        self.model_type = 'open_clip' if isinstance(vision_encoder, open_clip.model.VisionTransformer) else 'transformers'
        self.mlp = nn.Sequential(
            nn.Linear(encoder_output_dim, encoder_output_dim * 4), nn.GELU(),
            nn.Linear(encoder_output_dim * 4, mlp_output_dim))
        print(f"--- MLP сконфигурирован: {encoder_output_dim} -> {mlp_output_dim} ---")

    def forward(self, pixel_values):
        if self.model_type == 'open_clip':
            image_features = self.vision_encoder(pixel_values)
        else: # transformers
            image_features = self.vision_encoder(pixel_values=pixel_values).pooler_output
        return self.mlp(image_features)

# --- 4. ОСНОВНАЯ ФУНКЦИЯ ЗАПУСКА ---
def main():
    ENCODER_CHOICE = 'pubmedclip' # 'biomedclip' или 'pubmedclip'
    MLP_OUTPUT_DIM = 4096
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_PATHS = {
        'biomedclip': 'microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224',
        'pubmedclip': 'flaviagiammarino/pubmed-clip-vit-base-patch32'
    }

    print(f"\n--- 4. Запускаю пайплайн: {ENCODER_CHOICE} на slake ---")

    if ENCODER_CHOICE == 'biomedclip':
        print("--- Загрузка BiomedCLIP... ---")
        model_obj, _, image_processor = open_clip.create_model_and_transforms('ViT-B-16', pretrained='openai')
        checkpoint_path = hf_hub_download(repo_id=MODEL_PATHS['biomedclip'], filename="open_clip_pytorch_model.bin")
        model_obj.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE), strict=False)
        vision_encoder = model_obj.visual
        encoder_output_dim = model_obj.visual.proj.shape[1]
        print(f"--- BiomedCLIP загружен. Фактическая размерность выхода: {encoder_output_dim} ---")
    else: # pubmedclip
        print("--- Загрузка PubMedCLIP... ---")
        vision_encoder = CLIPVisionModel.from_pretrained(MODEL_PATHS['pubmedclip'])
        image_processor = AutoImageProcessor.from_pretrained(MODEL_PATHS['pubmedclip'])
        encoder_output_dim = vision_encoder.config.hidden_size
        print(f"--- PubMedCLIP загружен. Фактическая размерность выхода: {encoder_output_dim} ---")

    dataset = SlakeDataset(image_processor=image_processor)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

    model = VisionMLP(
        vision_encoder=vision_encoder,
        encoder_output_dim=encoder_output_dim,
        mlp_output_dim=MLP_OUTPUT_DIM
    ).to(DEVICE)

    for param in model.vision_encoder.parameters(): param.requires_grad = False

    print("\n--- 5. Обрабатываю один батч... ---")
    model.eval()
    with torch.no_grad():
        batch = next(iter(dataloader))
        images = batch['image'].to(DEVICE)
        image_embeddings = model(images)

        print(f"\n- Форма входного батча: {images.shape}")
        print(f"- Форма выходных эмбеддингов: {image_embeddings.shape}")
        assert image_embeddings.shape[1] == MLP_OUTPUT_DIM, "Размерность эмбеддинга неверна!"
        print("\n--- Проверка формы пройдена. ---")

    print("\n\n*** ПАЙПЛАЙН УСПЕШНО ВЫПОЛНЕН ***")

# --- 5. ЗАПУСК ---
if __name__ == '__main__':
    main()


--- 1. Установка правильных зависимостей (включая open_clip_torch) ---

--- 4. Запускаю пайплайн: pubmedclip на slake ---
--- Загрузка PubMedCLIP... ---
--- PubMedCLIP загружен. Фактическая размерность выхода: 768 ---


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

--- MLP сконфигурирован: 768 -> 4096 ---

--- 5. Обрабатываю один батч... ---

- Форма входного батча: torch.Size([4, 3, 224, 224])
- Форма выходных эмбеддингов: torch.Size([4, 4096])

--- Проверка формы пройдена. ---


*** ПАЙПЛАЙН УСПЕШНО ВЫПОЛНЕН ***
